###IMPORTS

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import sklearn
import nltk
np.random.seed(0)

###READING THE DATA

In [ ]:
p = pd.read_csv("/content/negative.review")
n = pd.read_csv('/content/positive.review')

In [ ]:
p.head()

,avid:1 your:1 horrible_book:1 wasted:1 use_it:1 the_entire:1 money.i:1 i_lit:1 i_read:1 lit:1 i_would:1 relationship:1 read:1 a_<num>:1 reader_and:1 reader:1 suffering:1 fire_one:1 i_had:1 year_old:2 gotten:1 horrible:3 lit_this:1 world...don't:1 my:2 one_star:1 headache_the:1 this_book:5 mom:1 was_horrible:1 friend:1 book_horrible:1 star_i:1 back:1 avid_reader:1 than_one:1 life:1 copy:1 rate_it:1 rate:1 my_mom:1 man:1 book_was:1 half:1 on_fire:1 and_then:1 reading_this:1 so:1 lower:1 i_could:1 <num>_year:2 than:1 time:2 half_of:1 time_spent:1 then:1 book:6 and_picked:1 possible:1 spent:1 old_man:1 up_after:1 one:2 horrible_if:1 one_less:1 part:1 was:2 entire:1 less_copy:1 to_rate:1 my_life:1 about_the:1 your_money.i:1 an_avid:1 if:1 the_relationship:1 use:1 a_headache:1 fire:1 lower_than:1 reading:1 a_friend:1 picked:1 purposes:1 then_got:1 waste_your:1 after_my:1 friend_i:1 old:2 man_and:1 and_i:1 world...don't_waste:1 book_on:1 part_about:1 copy_in:1 book_back:1 book_wasted:1 have_i:1 time_and:1 the_world...don't:1 better:1 if_it:1 star:1 got:1 mom_had:1 read_half:1 waste:1 after:1 i:6 about:1 could_use:1 had_gotten:1 was_possible:1 year:2 it_lower:1 relationship_the:1 wasted_my:1 wish:1 wish_i:1 boy:1 purposes_this:1 got_to:1 the_time:1 it_was:1 back_so:1 suffering_from:1 spent_reading:1 book_up:1 less:1 better_purposes:1 headache:1 possible_to:1 money.i_wish:1 for_better:1 it_suffering:1 the_part:1 gotten_it:1 picked_this:1 entire_time:1 old_boy:1 i_am:1 the_<num>:1 boy_had:1 <num>:2 so_i:1 #label#:negative
0,to_use:1 shallow:1 found:1 he_castigates:1 cas...
1,avid:1 your:1 horrible_book:1 wasted:1 use_it:...
2,book_seriously:1 we:1 days_couldn't:1 me_tell:...
3,"mass:1 only:1 he:2 help:1 ""jurisfiction"":1 lik..."
4,save_your:1 class_and:1 his_facts:1 opinions:1...


###PREPROCESSING

In [ ]:
#sentiment_reader.py execution
import codecs
import numpy as np

class SentimentCorpus:
    
    def __init__(self, train_per=0.8, dev_per=0, test_per=0.2):
        '''
        prepare dataset
        1) build feature dictionaries
        2) split data into train/dev/test sets 
        '''
        X, y, feat_dict, feat_counts = build_dicts()
        self.nr_instances = y.shape[0]
        self.nr_features = X.shape[1]
        self.X = X
        self.y = y
        self.feat_dict = feat_dict
        self.feat_counts = feat_counts
        
        train_y, dev_y, test_y, train_X, dev_X, test_X = split_train_dev_test(self.X, self.y, train_per, dev_per, test_per)
        self.train_X = train_X
        self.train_y = train_y
        self.dev_X = dev_X
        self.dev_y = dev_y
        self.test_X = test_X
        self.test_y = test_y

def split_train_dev_test(X, y, train_per, dev_per, test_per):
    if (train_per + dev_per + test_per) > 1:
        print ("train/dev/test splits should sum to one")
        return
    dim = y.shape[0]
    split1 = int(dim * train_per)
    
    if dev_per == 0:
        train_y, test_y = np.vsplit(y, [split1])
        dev_y = np.array([])
        train_X = X[0:split1,:]
        test_X = X[split1:,:]
        dev_X = np.array([])
    else:
        split2 = int(dim*(train_per+dev_per))
        train_y,dev_y,test_y = np.vsplit(y,(split1,split2))
        train_X = X[0:split1,:]
        dev_X = X[split1:split2,:]
        test_X = X[split2:,:]
        
    return train_y,dev_y,test_y,train_X,dev_X,test_X

def build_dicts():
    '''
    builds feature dictionaries
    ''' 
    feat_counts = {}

    # build feature dictionary with counts
    nr_pos = 0
    with codecs.open("positive.review", 'r', 'utf8') as pos_file:
        for line in pos_file:
            nr_pos += 1
            toks = line.split(" ")
            for feat in toks[0:-1]:
                name, counts = feat.split(":")
                if name not in feat_counts:
                    feat_counts[name] = 0
                feat_counts[name] += int(counts)
    
    nr_neg = 0
    with codecs.open("negative.review", 'r', 'utf8') as neg_file:
        for line in neg_file:
            nr_neg += 1
            toks = line.split(" ")
            for feat in toks[0:-1]:
                name, counts = feat.split(":")
                if name not in feat_counts:
                    feat_counts[name] = 0
                feat_counts[name] += int(counts)

    # remove all features that occur less than 5 (threshold) times
    to_remove = []
    for key, value in feat_counts.items():
        if value < 5:
            to_remove.append(key)
    for key in to_remove:
        del feat_counts[key]

    # map feature to index
    feat_dict = {}
    i = 0
    for key in feat_counts.keys():
        feat_dict[key] = i
        i += 1

    nr_feat = len(feat_counts) 
    nr_instances = nr_pos + nr_neg
    X = np.zeros((nr_instances, nr_feat), dtype=float)
    y = np.vstack((np.zeros([nr_pos,1], dtype=int), np.ones([nr_neg,1], dtype=int)))
    
    with codecs.open("positive.review", 'r', 'utf8') as pos_file:
        nr_pos = 0
        for line in pos_file:
            toks = line.split(" ")
            for feat in toks[0:-1]:
                name, counts = feat.split(":")
                if name in feat_dict:
                    X[nr_pos,feat_dict[name]] = int(counts)
            nr_pos += 1
        
    with codecs.open("negative.review", 'r', 'utf8') as neg_file:
        nr_neg = 0
        for line in neg_file:
            toks = line.split(" ")
            for feat in toks[0:-1]:
                name, counts = feat.split(":")
                if name in feat_dict:
                    X[nr_pos+nr_neg,feat_dict[name]] = int(counts)
            nr_neg += 1
    
    # shuffle the order, mix positive and negative examples
    new_order = np.arange(nr_instances)
    np.random.seed(0) # set seed
    np.random.shuffle(new_order)
    X = X[new_order,:]
    y = y[new_order,:]
    
    return X, y, feat_dict, feat_counts



In [ ]:
a = SentimentCorpus()

In [ ]:
print(a.train_X.shape)

(1600, 13989)


In [ ]:
print(a.test_y.shape)

(400, 1)


In [ ]:
pclass_train=0
nclass_train=0
pclass_test=0
nclass_test=0
for x in range (0,1600):
  if a.train_y[x] == 0:
    pclass_train = pclass_train+1
  else:
    nclass_train = nclass_train+1  
for x in range (0,400):
  if a.test_y[x]== 0:
     pclass_test = pclass_test+1
  else:
    nclass_test = nclass_test+1     

In [ ]:
print("number of positive and negative in training data:", pclass_train,nclass_train)
print("number of positive and negative in testing data:", pclass_test,nclass_test)

number of positive and negative in training data: 788 812
number of positive and negative in testing data: 212 188


In [ ]:
print('total number of records:',a.nr_instances)
print("total number of words in vocabulary:",a.nr_features)

total number of records: 2000
total number of words in vocabulary: 13989


###MODELLING

In [ ]:
#Importing MLPClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import accuracy_score,precision_score,recall_score,f1_score

In [ ]:
#calculate the various evaluation metrics
def metrics(pred):
  y_pred = pred
  acc = accuracy_score(a.test_y,y_pred)
  p = precision_score(a.test_y,y_pred)
  r = recall_score(a.test_y,y_pred)
  f1 = f1_score(a.test_y,y_pred)
  return acc,p,r,f1

In [ ]:
#using gridsearch to find the best combination of hyper parameters
from sklearn.model_selection import GridSearchCV
params = { 
    "hidden_layer_sizes" : [(100),(100,70),(80,150,90)], #one, two and three layers with different hidden layer sizes
    "max_iter":[100], #number of training epochs
    "activation":['relu','tanh','logistic'] #three different activation functions
}


In [ ]:
a.train_y = a.train_y.ravel()
a.test_y = a.test_y.ravel

In [ ]:
nn_model = MLPClassifier() #loading the Multi layer perceptron classifier model
g = GridSearchCV(nn_model,params,n_jobs= -1,cv=5) 
#gridsearch with custom parameters, model instance and cross fold validation as 5
g.fit(a.train_X,a.train_y)#fitting the model with the train and test data
scores=g.cv_results_['mean_test_score']
print(scores)
g.best_params_#finding the best combination paramerters for best accuracy values

[0.825625 0.83625  0.820625 0.8325   0.82875  0.815625 0.825    0.828125
 0.818125]


{'activation': 'relu', 'hidden_layer_sizes': (100, 70), 'max_iter': 100}

In [ ]:
#using the best parameter values to get the evaluation metric values
nn_model = MLPClassifier(hidden_layer_sizes=(80,150,90), max_iter=100,activation = 'relu' ,solver='adam',random_state=1)
nn_model.fit(a.train_X,a.train_y)
y_pred = nn_model.predict(a.test_X)
p,q,r,s = metrics(y_pred)
print( "accuracy score:", p)
print( "precision score:", q)
print( "recall score :", r)
print( "f1 score:", s)


/usr/local/lib/python3.8/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:1109: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


accuracy score: 0.85
precision score: 0.8076923076923077
recall score : 0.8936170212765957
f1 score: 0.8484848484848485
